# Diederske grupe 
Študiramo (2d) upodobitve diederskih grup
 $D_{2n} = <r, s \mid r^n = s^2 = (rs)^2 = 1>$.

### Diferencial $\mathcal L_{\text{rel}}$
$$
 L_{\text{rel}} = \frac{1}{3}(||\rho(s)^2 - I ||^2_F + ||\rho(r)^n -I||^2_F + || (\rho(r)\rho(s))^2 - I ||^2_F) 
$$
Veljajo pa enakosti
$$
\frac{d}{dA}||A^n - I||^2_F = 2(A^n - I) \sum \limits_{k=0}^{n-1}A^k \otimes A^{n - 1 - k}
$$
in
$$
\frac{d}{dA}||(AB)^2 - I||^2_F = 2(A^n - I)\left (I \otimes (AB) + (AB) \otimes I\right )(B^T \otimes I)
$$

Tako je TODO 

In [1]:
# definiraj grad_L

In [2]:
# poženi solve_ivp

In [5]:
# graf karakterjev